In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from autodm.llm import get_llm
from autodm.roll import Dice, DiceType
from autodm.character import Character
from autodm.locaitons import Location, LocationStore

from rich import print

In [17]:
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core import PromptTemplate
from pydantic import BaseModel, Field
from typing import Optional, List, Any

In [3]:
character = Character.generate(level=0)
print(character)

In [9]:
region = Location.generate(type="region")
locations = LocationStore(region, **{region.name: region})
print(locations)

{
    'The Kingdom of Elvendom': Location(
        type='region',
        name='The Kingdom of Elvendom',
        description='A vast, lush kingdom ruled by the elven people, filled with ancient forests and crystal-clear 
lakes. The elves are known for their mastery of magic and archery, and they live in harmony with nature.',
        parent_name=None,
        parent_description=None
    )
}

In [12]:
city = locations.current.generate_child(type="city")
while city.type != "city":
    city = locations.current.generate_child(type="city")
locations.add(city)
locations.set_current(city)

In [13]:
print(locations)

{
    'The Kingdom of Elvendom': Location(
        type='region',
        name='The Kingdom of Elvendom',
        description='A vast, lush kingdom ruled by the elven people, filled with ancient forests and crystal-clear 
lakes. The elves are known for their mastery of magic and archery, and they live in harmony with nature.',
        parent_name=None,
        parent_description=None
    ),
    'Elvendale': Location(
        type='city',
        name='Elvendale',
        description='A beautiful and bustling elven city nestled in the heart of the Kingdom of Elvendom, known for
its grand architecture and enchanting atmosphere. The city is a hub of trade and culture, with a diverse population
of elves from all across the kingdom.',
        parent_name='The Kingdom of Elvendom',
        parent_description='A vast, lush kingdom ruled by the elven people, filled with ancient forests and 
crystal-clear lakes. The elves are known for their mastery of magic and archery, and they live in harmony with 
nature.'
    )
}

In [29]:
class StoryItem(BaseModel):
    "A story item for a D&D campaign."
    name: str = Field(..., title="Name", description="The name of the story item.")
    details: str = Field(..., title="Description", description="A description of the story item.")

class StoryItems(BaseModel):
    items: List[StoryItem] = Field([], title="Items", description="The story items in the storyline.")

class Storyline(BaseModel):
    "A storyline for a D&D campaign. Contains the overarching story for a one-shot campaign."
    name: str = Field(..., title="Name", description="The name of the storyline.")
    details: str = Field(..., title="Description", description="A description of the storyline.")
    items: Optional[List[StoryItem]] = Field([], title="Items", description="The story items in the storyline.")

    @classmethod
    def generate(cls, character, city, region, locations):
        start_story_template = PromptTemplate("""\
You are a dungeion master creating a storyline for a D&D one-shot campaign. \
Here are the characters in the party: 
{characters}
                                      
The party is currently in the {current_location} of the {region} region. \
Here is the detailed location information:
{locations}
                                      
Please create the name and details for the storyline. Do not generate any story items yet. \
                                      
Answer: \
""").partial_format(characters=[character], current_location=city.name, region=region.name, locations=locations)
        storyline_program = LLMTextCompletionProgram.from_defaults(llm=get_llm(), prompt=start_story_template, output_cls=Storyline)
        storyline = storyline_program()
        
        return program()
    
    def generate_story_items(self):
        start_story_item_template = PromptTemplate("""\
You are a dungeion master creating a story item for a D&D quest. \
Here is the storyline you are working on:
Title: {name}
Description: {details}
                                                   
Create 3-5 events that will happen in the storyline if it's followed. \
The final event should be the conclusion of the storyline. \
""")

In [28]:
Storyline.generate(character, city, region, locations)

Storyline(name='The Quest for the Elvendale Crystal', details="The Kingdom of Elvendom is facing an imminent threat as the powerful Elvendale Crystal, which grants magical energy to the entire realm, has gone missing. The crystal's absence causes the land to wither and lose its luster, and without it, the elves' magic fades.", items=[StoryItem(name='The Lost Crystal', details='The Elvendale Crystal, a large gemstone imbued with powerful arcane energy, has vanished without a trace. It is said that a group of dark creatures from the neighboring realm have stolen it to harness its power for their own nefarious purposes.'), StoryItem(name='The Dark Creatures', details='A band of goblins and orcs led by a powerful necromancer has taken the crystal, using its energy to resurrect their fallen comrades and create an army of undead. They are marching towards Elvendale with the intent to conquer it.'), StoryItem(name='The Journey', details='To reclaim the crystal, the party must embark on a trea

In [19]:
start_story_template = PromptTemplate("""\
You are a dungeion master creating a storyline for a D&D one-shot campaign. \
Here are the characters in the party: 
{characters}
                                      
The party is currently in the {current_location} of the {region} region. \
Here is the detailed location information:
{locations}
                                      
Create 3-5 major story items for the storyline. \
Each story item should have a name and a description. \
When the storyline is complete, the quest should be achieved.\
                                      
Storyline: \
""").partial_format(characters=[character], current_location=city.name, region=region.name, locations=locations)

In [20]:
llm = get_llm()

In [23]:
program = LLMTextCompletionProgram.from_defaults(llm=get_llm(), prompt=start_story_template, output_cls=Storyline)

In [24]:
program()

ValidationError: 1 validation error for Storyline
details
  Field required [type=missing, input_value={'name': 'The Quest for t...dawns upon the land.'}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.7/v/missing

In [4]:
from llama_index.core.program import LLMTextCompletionProgram
from pydantic import BaseModel, Field
from typing import Optional, List, Any